In [2]:
import os
import pandas as pd
from corrLib import readdata
from utils import *
import numpy as np

## See the current data

Run the cell below to know what has or has not been done!

In [3]:
data_folder = '../Data'
log = pd.read_csv(os.path.join(data_folder, 'main_log.csv'), header=[0,1])
log#.head() # to see the whole table, comment out the .head()

params                                                              \
       Date                                     Subfolder  BC   MPP FPS   
0   7132021                          2021-07-13_17h17m24s  20  0.10  30   
1   7202021  1st-drop-tracking/2021-07-20_16h24m06s-mttry  32  0.10  30   
2   7202021     1st-drop-tracking/2021-07-20_16h25m35s-m2  32  0.10  30   
3   7202021     1st-drop-tracking/2021-07-20_16h27m01s-m3  32  0.10  30   
4   7202021     1st-drop-tracking/2021-07-20_16h28m21s-m4  32  0.10  30   
5   7202021     1st-drop-tracking/2021-07-20_16h29m43s-m5  32  0.10  30   
6   7202021        1st-drop-tracking/2021-07-20_16h31m06s  32  0.10  30   
7   7202021        1st-drop-tracking/2021-07-20_16h32m59s  32  0.10  30   
8   7202021        1st-drop-tracking/2021-07-20_16h34m28s  32  0.10  30   
9   7202021        1st-drop-tracking/2021-07-20_16h35m47s  32  0.10  30   
10  7202021        1st-drop-tracking/2021-07-20_16h42m46s  32  0.10  30   
11  7202021        1st-drop-tracking/2021-07-20_16h43m54s  32  0.10  30   
12  7202021        1st-drop-tracking/2021-07-20_16h44m58s  32  0.10  30   
13  7202021        1st-drop-tracking/2021-07-20_16h47m59s  32  0.10  30   
14  7202021        1st-drop-tracking/2021-07-20_16h50m22s  32  0.10  30   
15  7202021        1st-drop-tracking/2021-07-20_16h51m25s  32  0.10  30   
16  7212021                          2021-07-21_16h41m52s  23  0.16  30   
17  7212021                          2021-07-21_16h43m03s  23  0.16  30   
18  8042021                          2021-08-04_16h32m08s  24  0.10  30   
19  8042021                          2021-08-04_16h37m16s  24  0.10  30   
20  8042021                          2021-08-04_16h42m12s  24  0.10  30   
21  8042021                          2021-08-04_16h43m56s  24  0.10  30   

   analysis                                                      Visual  \
         ID     OD     a     b   c 2D-trajectory 3D-trajectory 2D-image   
0      34.5  100.5  45.4  49.0 NaN             1             1        0   
1       8.8   66.0  51.4  51.2 NaN             1             1        1   
2       9.0   69.5  51.3  52.0 NaN             1             1        1   
3       9.0   69.2  50.9  51.8 NaN             1             1        1   
4       9.1   69.3  51.2  51.2 NaN             1             1        1   
5       NaN    NaN   NaN   NaN NaN             1             0        0   
6       NaN    NaN   NaN   NaN NaN             1             0        0   
7       NaN    NaN   NaN   NaN NaN             1             0        0   
8       NaN    NaN   NaN   NaN NaN             1             0        0   
9       NaN    NaN   NaN   NaN NaN             1             0        0   
10      8.4   71.7  51.8  51.4 NaN             1             1        0   
11      NaN    NaN   NaN   NaN NaN             1             0        0   
12      NaN    NaN   NaN   NaN NaN             1             0        0   
13      NaN    NaN   NaN   NaN NaN             1             0        0   
14      NaN    NaN   NaN   NaN NaN             1             0        0   
15      NaN    NaN   NaN   NaN NaN             1             1        0   
16     11.1   50.4  36.2  29.4 NaN             1             1        0   
17      NaN    NaN   NaN   NaN NaN             0             0        0   
18      NaN    NaN   NaN   NaN NaN             0             0        0   
19      NaN    NaN   NaN   NaN NaN             0             0        0   
20      NaN    NaN   NaN   NaN NaN             0             0        0   
21      NaN    NaN   NaN   NaN NaN             0             0        0   

                            
   3D-projection 3D-outRef  
0              0         0  
1              1         1  
2              1         1  
3              1         1  
4              1         1  
5              0         0  
6              0         0  
7              0         0  
8              0         0  
9              0         0  
10             1         1  
11             0         0

## Update the log

- frame rate from RawImageInfo.txt
- check the dropSize.xlsx for a, b, ID and OD
- check traj.csv and xyz-traj.csv for 2D and 3D trajectory
- check key visualization files
    - 2D-image: "2d-traj.jpg"
    - 3D-projection: "3d-traj-animation.avi"
    - 3D-outRef: "3d-traj-RVRef.avi"

In [4]:
img_folder = '/home/zhengyang/data/DE'

for num, i in log.iterrows():
    sf = os.path.join(img_folder, '{:08d}'.format(i[('params', 'Date')]), i[('params', 'Subfolder')])
    # RawImageInfo.txt
    imageInfo_dir = os.path.join(sf, 'RawImageInfo.txt')
    if os.path.exists(imageInfo_dir) == True:
        with open(imageInfo_dir, 'r') as f:
            fps = int(f.readline())
        log[('params', 'FPS')].at[num] = fps

    # dropSize.xlsx
    dropSize_dir = os.path.join(sf, 'dropSize.xlsx')
    if os.path.exists(dropSize_dir) == True:
        outer, inner, a, b = read_dropSize(dropSize_dir)
        log[('analysis', 'OD')].at[num] = outer
        log[('analysis', 'ID')].at[num] = inner
        log[('analysis', 'a')].at[num] = a
        log[('analysis', 'b')].at[num] = b
    
    # traj.csv
    traj_dir = os.path.join(sf, 'crop_HoughCircles', 'traj.csv')
    log[('analysis', '2D-trajectory')].at[num] = int(os.path.exists(traj_dir))
    # xyz-traj.csv
    xyz_dir = os.path.join(sf, 'crop_HoughCircles', 'xyz-traj.csv')
    log[('analysis', '3D-trajectory')].at[num] = int(os.path.exists(xyz_dir))
    # 2d-image.jpg
    image_2D_dir = os.path.join(sf, 'crop_HoughCircles', '2d-image.jpg')
    log[('Visual', '2D-image')].at[num] = int(os.path.exists(image_2D_dir))
    # 3d-traj-animation.avi
    projection_3D_dir = os.path.join(sf, 'crop_HoughCircles', '3d-traj-animation.avi')
    log[('Visual', '3D-projection')].at[num] = int(os.path.exists(projection_3D_dir))
    # 3d-traj-RVRef.avi
    outRef_3D_dir = os.path.join(sf, 'crop_HoughCircles', '3d-traj-RVRef.avi')
    log[('Visual', '3D-outRef')].at[num] = int(os.path.exists(outRef_3D_dir))    

In [5]:
log.head()

params                                                             \
      Date                                     Subfolder  BC  MPP FPS   
0  7132021                          2021-07-13_17h17m24s  20  0.1  30   
1  7202021  1st-drop-tracking/2021-07-20_16h24m06s-mttry  32  0.1  30   
2  7202021     1st-drop-tracking/2021-07-20_16h25m35s-m2  32  0.1  30   
3  7202021     1st-drop-tracking/2021-07-20_16h27m01s-m3  32  0.1  30   
4  7202021     1st-drop-tracking/2021-07-20_16h28m21s-m4  32  0.1  30   

  analysis                                                      Visual  \
        ID     OD     a     b   c 2D-trajectory 3D-trajectory 2D-image   
0     34.5  100.5  45.4  49.0 NaN             1             1        0   
1      8.8   66.0  51.4  51.2 NaN             1             1        1   
2      9.0   69.5  51.3  52.0 NaN             1             1        1   
3      9.0   69.2  50.9  51.8 NaN             1             1        1   
4      9.1   69.3  51.2  51.2 NaN             1             1        1   

                           
  3D-projection 3D-outRef  
0             0         0  
1             1         1  
2             1         1  
3             1         1  
4             1         1

## Save the log

In [26]:
log.to_csv(os.path.join(data_folder, 'main_log.csv'), index=False)

## Add entries to log

In [19]:
folder = '/home/zhengyang/data/DE/07202021'
l = readdata(folder, 'raw')
for num, i in l.iterrows():
    print(os.path.relpath(os.path.split(i.Dir)[0], folder))
# copy to the csv spreadsheet

1st-drop-tracking/2021-07-20_16h47m59s
1st-drop-tracking/2021-07-20_16h50m22s
1st-drop-tracking/2021-07-20_16h44m58s
1st-drop-tracking/2021-07-20_16h35m47s
1st-drop-tracking/2021-07-20_16h34m28s
1st-drop-tracking/2021-07-20_16h31m06s
1st-drop-tracking/2021-07-20_16h28m21s-m4
1st-drop-tracking/2021-07-20_16h25m35s-m2
1st-drop-tracking/2021-07-20_16h27m01s-m3
1st-drop-tracking/2021-07-20_16h29m43s-m5
1st-drop-tracking/2021-07-20_16h32m59s
1st-drop-tracking/2021-07-20_16h24m06s-mttry
1st-drop-tracking/2021-07-20_16h43m54s
1st-drop-tracking/2021-07-20_16h51m25s
1st-drop-tracking/2021-07-20_16h42m46s
